# Amazon Fashion Binary

Il progetto è volto allo sviluppo di un modello ML per svolgere sentiment analysis su recensioni di utenti amazon al fine di classificarle come positive o negative.

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import models, layers

## Dataset
Il dataset è composto da 20000 recensioni positive e 20000 recensioni negative. Verranno provate varie configurazioni del dataset: la prima consiste nel dataset completo senza apportare nessuna modifica alle recensioni, nella seconda le recensioni verranno modificate in base alla loro lunghezza e alla dimensione del vocabolario.

In [2]:
#path della directory deel dataset
directory_path = "./reviews"

Il dataset viene dapprima diviso in training set (70%), validation set (15%) e test set (15%)

In [3]:
train_dataset, test_validation_dataset = tf.keras.preprocessing.text_dataset_from_directory(
    directory_path,
    labels='inferred',
    label_mode='int',
    class_names=None,
    batch_size=32,
    max_length=None,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset="both",
    follow_links=False,
    verbose=True
)
test_dataset, validation_dataset = tf.keras.utils.split_dataset(
    test_validation_dataset, left_size=0.5, shuffle=True, seed=42
)
X_train = train_dataset.map(lambda x, y: x)
X_test = test_dataset.map(lambda x, y: x)
X_validation = validation_dataset.map(lambda x, y: x)

Found 40000 files belonging to 2 classes.
Using 28000 files for training.
Using 12000 files for validation.


Vengono calcolate la media e la varianza della lunghezza delle recensioni: questi dati verranno utilizzati per scegliere le configurazioni del dataset su cui il modello opererà.

In [4]:
#lunghezza media delle recensioni
DATASET_SIZE = 40000
len_strings = []

for batch in X_train:
    n = batch.numpy()
    for data in n:
        split = data.split()
        len_strings.append(len(split))

for batch in X_test:
    n = batch.numpy()
    for data in n:
        split = data.split()
        len_strings.append(len(split))

for batch in X_validation:
    n = batch.numpy()
    for data in n:
        split = data.split()
        len_strings.append(len(split))

len_mean = sum(len_strings) / DATASET_SIZE
len_variance = sum((len_text - len_mean) ** 2 for len_text in len_strings) / DATASET_SIZE

print("Media: " + str(len_mean) + "\nVarianza: " + str(len_variance))

Media: 33.1579
Varianza: 1501.74106759


# Prima configurazione

Nella tokenizzazione delle recensione vengono scelte una size per il vocabolario pari a 30 e una lunghezza massima delle recensioni di 20 (sotto la media).

In [5]:
MAX_SEQUENCE_LENGTH = 20
VOCAB_SIZE = 30
vectorizationLayer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)
vectorizationLayer.adapt(X_train)

In [6]:
model = models.Sequential()

In [7]:
# Input - Layer
model.add(vectorizationLayer)
# Hidden - Layers
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ ?                      │   0 (unbuilt) │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(
optimizer = "adam",
loss = "binary_crossentropy",
metrics = ["accuracy"]
)

In [9]:
model.fit(
train_dataset,
epochs= 20,
batch_size = 32,
validation_data = validation_dataset
)

Epoch 1/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5307 - loss: 0.8173 - val_accuracy: 0.5470 - val_loss: 0.6885
Epoch 2/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.5447 - loss: 0.6921 - val_accuracy: 0.5511 - val_loss: 0.6853
Epoch 3/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.5538 - loss: 0.6870 - val_accuracy: 0.5531 - val_loss: 0.6829
Epoch 4/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.5693 - loss: 0.6820 - val_accuracy: 0.5617 - val_loss: 0.6802
Epoch 5/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.5705 - loss: 0.6782 - val_accuracy: 0.5652 - val_loss: 0.6787
Epoch 6/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.5721 - loss: 0.6780 - val_accuracy: 0.5683 - val_loss: 0.6771
Epoch 7/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.5747 - loss: 0.6759 - val_accuracy: 0.5642 - val_loss: 0.6782
Epoch 8/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.5799 - loss: 0.6744 - val_accuracy: 0

In [10]:
loss, accuracy = model.evaluate(train_dataset)
print(f"Training set:\nLoss: {loss}, Accuracy: {accuracy}")
loss, accuracy = model.evaluate(test_dataset)
print(f"Test set:\nLoss: {loss}, Accuracy: {accuracy}")

875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.6039 - loss: 0.6602
Training set:
Loss: 0.6578472852706909, Accuracy: 0.607285737991333
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5939 - loss: 0.6671
Test set:
Loss: 0.6672678589820862, Accuracy: 0.5885970592498779


## Seconda configurazione

Nella tokenizzazione delle recensione vengono scelte una size per il vocabolario pari a 50 e una lunghezza massima delle recensioni di 35 (circa la media).


In [11]:
MAX_SEQUENCE_LENGTH = 35
VOCAB_SIZE = 50
vectorizationLayer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)
vectorizationLayer.adapt(X_train)

In [12]:
model = models.Sequential()

In [13]:
# Input - Layer
model.add(vectorizationLayer)
# Hidden - Layers
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization_1            │ ?                      │   0 (unbuilt) │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(
optimizer = "adam",
loss = "binary_crossentropy",
metrics = ["accuracy"]
)

In [15]:
model.fit(
train_dataset,
epochs= 20,
batch_size = 32,
validation_data = validation_dataset
)

Epoch 1/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.5246 - loss: 0.8644 - val_accuracy: 0.5505 - val_loss: 0.6810
Epoch 2/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5413 - loss: 0.6936 - val_accuracy: 0.5730 - val_loss: 0.6759
Epoch 3/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5656 - loss: 0.6822 - val_accuracy: 0.5705 - val_loss: 0.6767
Epoch 4/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5711 - loss: 0.6786 - val_accuracy: 0.5800 - val_loss: 0.6739
Epoch 5/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.5802 - loss: 0.6743 - val_accuracy: 0.5836 - val_loss: 0.6729
Epoch 6/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5804 - loss: 0.6750 - val_accuracy: 0.5767 - val_loss: 0.6733
Epoch 7/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5874 - loss: 0.6730 - val_accuracy: 0.5866 - val_loss: 0.6738
Epoch 8/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5894 - loss: 0.6714 - val_accuracy: 0.

In [16]:
loss, accuracy = model.evaluate(train_dataset)
print(f"Training set:\nLoss: {loss}, Accuracy: {accuracy}")
loss, accuracy = model.evaluate(test_dataset)
print(f"Test set:\nLoss: {loss}, Accuracy: {accuracy}")

875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.6297 - loss: 0.6451
Training set:
Loss: 0.6411459445953369, Accuracy: 0.6331071257591248
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5883 - loss: 0.6690
Test set:
Loss: 0.6707727909088135, Accuracy: 0.5884308218955994


## Terza configurazione

Nella tokenizzazione delle recensione vengono scelte una size per il vocabolario pari a 100 e una lunghezza massima delle recensioni di 70 (sopra la media).


In [17]:
MAX_SEQUENCE_LENGTH = 70
VOCAB_SIZE = 100
vectorizationLayer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)
vectorizationLayer.adapt(X_train)

In [18]:
model = models.Sequential()

In [19]:
# Input - Layer
model.add(vectorizationLayer)
# Hidden - Layers
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization_2            │ ?                      │   0 (unbuilt) │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(
optimizer = "adam",
loss = "binary_crossentropy",
metrics = ["accuracy"]
)

In [21]:
model.fit(
train_dataset,
epochs= 20,
batch_size = 32,
validation_data = validation_dataset
)

Epoch 1/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5233 - loss: 1.1825 - val_accuracy: 0.5396 - val_loss: 0.6857
Epoch 2/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.5457 - loss: 0.6990 - val_accuracy: 0.5545 - val_loss: 0.6774
Epoch 3/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5484 - loss: 0.6893 - val_accuracy: 0.5662 - val_loss: 0.6757
Epoch 4/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5622 - loss: 0.6798 - val_accuracy: 0.5757 - val_loss: 0.6742
Epoch 5/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5646 - loss: 0.6773 - val_accuracy: 0.5650 - val_loss: 0.6748
Epoch 6/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5728 - loss: 0.6747 - val_accuracy: 0.5720 - val_loss: 0.6739
Epoch 7/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5827 - loss: 0.6725 - val_accuracy: 0.5745 - val_loss: 0.6739
Epoch 8/20
875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5845 - loss: 0.6709 - val_accuracy: 0

In [22]:
loss, accuracy = model.evaluate(train_dataset)
print(f"Training set:\nLoss: {loss}, Accuracy: {accuracy}")
loss, accuracy = model.evaluate(test_dataset)
print(f"Test set:\nLoss: {loss}, Accuracy: {accuracy}")

875/875 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.6407 - loss: 0.6387
Training set:
Loss: 0.6351535320281982, Accuracy: 0.6462500095367432
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5975 - loss: 0.6660
Test set:
Loss: 0.6692769527435303, Accuracy: 0.5844414830207825


## Conclusione

Le tre configurazioni hanno tutte prestazioni simili, sia testando il modello con i dati del training, sia con quelli del testing: le maggiori differenze si possono notare nell'accuracy per quanto riguarda la seconda e la terza configurazione, che presentano uno scarto di circa il 5% tra training e test set a favore del primo.
